In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e14/sample_submission.csv
/kaggle/input/playground-series-s3e14/train.csv
/kaggle/input/playground-series-s3e14/test.csv


In [2]:
# reading train and test into pd files. 

X_train = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')
X_test = pd.read_csv('/kaggle/input/playground-series-s3e14/test.csv')

In [3]:
#  separate target from predictors
y = X_train['yield']              
X_train.drop(['yield'], axis=1, inplace=True)

In [4]:
X_train.head()

,id,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,0,25.0,0.50,0.25,0.75,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.425011,0.417545,32.460887
1,1,25.0,0.50,0.25,0.50,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.444908,0.422051,33.858317
2,2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.552927,0.470853,38.341781
3,3,12.5,0.25,0.25,0.63,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.565976,0.478137,39.467561
4,4,25.0,0.50,0.25,0.63,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.579677,0.494165,40.484512


In [5]:
y.head()

0    4476.81146
1    5548.12201
2    6869.77760
3    6880.77590
4    7479.93417
Name: yield, dtype: float64

In [6]:
X_train.dtypes, X_train.shape

(id                        int64
 clonesize               float64
 honeybee                float64
 bumbles                 float64
 andrena                 float64
 osmia                   float64
 MaxOfUpperTRange        float64
 MinOfUpperTRange        float64
 AverageOfUpperTRange    float64
 MaxOfLowerTRange        float64
 MinOfLowerTRange        float64
 AverageOfLowerTRange    float64
 RainingDays             float64
 AverageRainingDays      float64
 fruitset                float64
 fruitmass               float64
 seeds                   float64
 dtype: object,
 (15289, 17))

In [7]:
# Break off validation set from training data
from sklearn.model_selection import train_test_split
X_train_f, X_valid, y_train_f, y_valid = train_test_split(X_train, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
# As all numeric, no need to convert to one-hot. 
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Define the model
my_model_1 = XGBRegressor(random_state=0) 

# Fit the model
my_model_1.fit(X_train_f, y_train_f)

# Check your answer
predictions_1 = my_model_1.predict(X_valid) 

# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 378.7298562911574


In [9]:
testing = [1, 10, 100, 1000]
lr = [0.5, 0.1, 0.05, 0.01, 0.001]
for t in testing:
    for i in lr:
        my_model_3 = XGBRegressor(n_estimators=t, learning_rate=i, n_jobs=4) 

        # Fit the model
        my_model_3.fit(X_train_f, y_train_f) # Your code here

        # Get predictions
        predictions_3 = my_model_3.predict(X_valid)

        # Calculate MAE
        mae_3 = mean_absolute_error(predictions_3, y_valid)

        # Uncomment to print MAE
        print(f'Mean Absolute Error for learning rate {i} and estimators {t}:' , mae_3)

Mean Absolute Error for learning rate 0.5 and estimators 1: 3007.6803225918698
Mean Absolute Error for learning rate 0.1 and estimators 1: 5409.9235288066675
Mean Absolute Error for learning rate 0.05 and estimators 1: 5710.511511179004
Mean Absolute Error for learning rate 0.01 and estimators 1: 5950.981899309558
Mean Absolute Error for learning rate 0.001 and estimators 1: 6005.087733609347
Mean Absolute Error for learning rate 0.5 and estimators 10: 360.07251103971646
Mean Absolute Error for learning rate 0.1 and estimators 10: 2099.8106413278824
Mean Absolute Error for learning rate 0.05 and estimators 10: 3598.986634655732
Mean Absolute Error for learning rate 0.01 and estimators 10: 5436.23806143904
Mean Absolute Error for learning rate 0.001 and estimators 10: 5951.251451315851
Mean Absolute Error for learning rate 0.5 and estimators 100: 393.6612516952972
Mean Absolute Error for learning rate 0.1 and estimators 100: 359.2070452573935
Mean Absolute Error for learning rate 0.05 a

In [10]:
my_model_4 = XGBRegressor(n_estimators=100, learning_rate=0.05, n_jobs=4) 


# Fit the model
my_model_4.fit(X_train_f, y_train_f) # Your code here

# Get predictions
predictions_4 = my_model_4.predict(X_valid)

# Calculate MAE
mae_4 = mean_absolute_error(predictions_4, y_valid)

# Uncomment to print MAE
print(f'Mean Absolute Error:' , mae_4)


Mean Absolute Error: 356.10419411096404


In [11]:
X_test.head()

,id,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,15289,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.399367,0.408088,31.394569
1,15290,12.5,0.25,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.488048,0.442866,36.846956
2,15291,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.583379,0.487057,40.037644
3,15292,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.433014,0.422847,33.116091
4,15293,37.5,0.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.360996,0.388860,29.558019


In [12]:
# Generate test predictions 
preds_test = my_model_4.predict(X_test)

In [13]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'id': X_test.id,
                       'yield': preds_test})
output.to_csv('submission.csv', index=False)